-training is iterative
-guess -> calculate error -> collect error -> optimize using gradient descent

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26421880/26421880 [01:02<00:00, 420961.21it/s] 


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 102603.98it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:13<00:00, 315947.04it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 8650752.00it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



#hyperparameter: adjustable parameter to control optimization
-number of epochs: number times to iterate
-batch size: number of data samples propagated before parameter update
-learning rate: how much to update parameters at each batch/epoch(small-slow speed, large-unpredictable)

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

#epoch: each iteration of optimization loop
-train loop: iterate training dataset, try to converge to optimal parameter
-validation/test loop: iterate test dataset to check perfomance improvement

#loss function: measure degree of dissimilarity of result to target value. try to minimize
-nn.MSELoss(mean square error) for regression
-nn.NLLLoss(negative log likelihood) for classification
-nn.CrossEntropyLoss(nn.LogSoftmax + nn.NLLLoss)

In [3]:
loss_fn = nn.CrossEntropyLoss()

-optimization logic is in optimizer object
-SGD, ADAM, RMSProp

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#optimization step
-call optimizer.zero_grad() to reset gradients of parameters. default add up. prevent double-counting
-backpropagate loss with loss.backward(). deposit gradients of loss
-optimizer.step() to adjust parameters by gradients

In [8]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        #compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        #backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [6]:
def test_loop(dataloader, model, loss_fn):
    #set model to evaluation model
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    #evaluting with torch.no_grad() ensures no gradients are computed
    # + reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------
loss: 2.306038 [   64/60000]
loss: 2.293473 [ 6464/60000]
loss: 2.275476 [12864/60000]
loss: 2.271276 [19264/60000]
loss: 2.264346 [25664/60000]
loss: 2.228623 [32064/60000]
loss: 2.240273 [38464/60000]
loss: 2.203691 [44864/60000]
loss: 2.197651 [51264/60000]
loss: 2.168291 [57664/60000]
Test Error: 
 Accuracy: 51.2%, Avg loss: 2.167592 

Epoch 2
-------------------
loss: 2.177039 [   64/60000]
loss: 2.168090 [ 6464/60000]
loss: 2.114296 [12864/60000]
loss: 2.130540 [19264/60000]
loss: 2.084133 [25664/60000]
loss: 2.025871 [32064/60000]
loss: 2.052601 [38464/60000]
loss: 1.973790 [44864/60000]
loss: 1.975776 [51264/60000]
loss: 1.901736 [57664/60000]
Test Error: 
 Accuracy: 55.9%, Avg loss: 1.908343 

Epoch 3
-------------------
loss: 1.942655 [   64/60000]
loss: 1.913241 [ 6464/60000]
loss: 1.799619 [12864/60000]
loss: 1.836529 [19264/60000]
loss: 1.733528 [25664/60000]
loss: 1.683345 [32064/60000]
loss: 1.701296 [38464/60000]
loss: 1.599881 [44864/60000]
